In [1]:
import torch


from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_dense_adj
from torch.utils.data import DataLoader, random_split

from torch_geometric.nn.pool import global_mean_pool, global_max_pool, global_add_pool

In [2]:
from helpers import CVFConfigDataset

In [3]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [4]:
dataset = CVFConfigDataset(
    "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
)
# dataset = CVFConfigDataset(
#     "graph_1_config_rank_dataset.csv", "graph_1_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
dataset = CVFConfigDataset(
    "graph_5_config_rank_dataset.csv", "graph_5_edge_index.json", 9
)
train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
# data_loader = DataLoader(dataset, batch_size=128, shuffle=False)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        h = torch.relu(h)
        h = self.out(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, data_loader, epochs):
        # criterion = torch.nn.CrossEntrop yLoss()
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        edge_index = dataset.edge_index.t().to(device)
        for epoch in range(1, epochs + 1):
            avg_loss = 0
            count = 0
            for batch in data_loader:
                x = batch[0].to(device)
                y = batch[1].to(device)
                y = y.unsqueeze(0).reshape(-1, 1, 1).float()
                optimizer.zero_grad()
                out = self(x, edge_index)
                # print("output", out, "y", y)
                loss = criterion(out, y)
                avg_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Epoch", epoch, "| Loss:", avg_loss / count)


In [7]:
gnn = VanillaGNN(dataset.num_classes, 64, 1).to(device)
print(gnn)

gnn.fit(train_loader, epochs=25)

VanillaGNN(
  (gcn1): GCNConv(9, 64)
  (gcn2): GCNConv(64, 64)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
Epoch 1 | Loss: tensor(0.1851, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2 | Loss: tensor(0.0151, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 3 | Loss: tensor(0.0123, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 4 | Loss: tensor(0.0099, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 5 | Loss: tensor(0.0096, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 6 | Loss: tensor(0.0087, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 7 | Loss: tensor(0.0079, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 8 | Loss: tensor(0.0090, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 9 | Loss: tensor(0.0079, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 10 | Loss: tensor(0.0076, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 11 | Loss: tensor(0.0081, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 12 | Loss: tensor(0.0073, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 13 

In [8]:
# testing
torch.no_grad()
torch.set_printoptions(profile="full")

total_matched = 0
edge_index = dataset.edge_index.t().to(device)

for batch in test_loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    y = y.unsqueeze(0).reshape(-1, 1, 1).float()
    predicted = gnn(x, edge_index)
    # predicted = predicted.argmax(dim=1)
    predicted = torch.round(predicted)
    matched = (predicted == y).sum().item()
    total_matched += matched
    

print("Total matched", total_matched, "out of", len(test_dataset), "| Accuracy", round(total_matched/len(test_dataset) * 100, 4), "%")

Total matched 132652 out of 132861 | Accuracy 99.8427 %
